 # Vectorized deep neural network implemenation

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [3]:
def initialize_parameters_deep(layer_dims):
    """
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    """
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

In [5]:
def linear_forward(A, W, b):
    """
    linear part of a layer's forward propagation.
    """
    
    Z = np.dot(W, A) + b
    cache = (A, W, b) # cache created so back-prop more efficient
    
    return Z, cache

In [7]:
#relu activation function for hidden layers and sigmoid for output layer
def relu(Z):
    ""

def sigmoid(Z):
    ""
    

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    """
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)    
    cache = (linear_cache, activation_cache)
    return A, cache